## 1. Preprocesamiento del texto (`preprocess.py`)

El preprocesamiento del texto se realiza en el archivo preprocess.py, que toma como entrada un archivo de texto (por ejemplo, un chat de WhatsApp) y lo transforma en un formato que el modelo puede usar. Primero, se lee el archivo línea por línea, se limpian caracteres poco útiles (como ciertos emojis o símbolos extraños) y, mediante expresiones regulares, se separa cada línea en componentes como fecha, contacto y mensaje, de manera que quede claro quién habló y qué dijo.

Después, se aplica una tokenización personalizada: el texto se descompone en unidades más pequeñas (tokens) como palabras, números o signos de puntuación, respetando algunos tokens especiales (por ejemplo, nombres de contacto o el marcador <END>). A partir de todos los tokens que aparecen en el corpus se construye un vocabulario ordenado, asignando a cada token un índice entero; luego, cada token del texto se reemplaza por su índice correspondiente, obteniendo un gran vector de números.

Por último, este vector se divide en dos partes: una porción grande se destina al conjunto de entrenamiento y una parte más pequeña al conjunto de validación. Ambos se guardan como tensores de PyTorch (por ejemplo train.pt y valid.pt), junto con el vocabulario, de modo que el modelo pueda trabajar directamente con estos datos numéricos en las siguientes etapas.


## 2. Tokenización, vocabulario y codificación (`utils.py`)

El archivo `utils.py` contiene funciones auxiliares que relacionan texto con índices numéricos.

Las funciones más importantes son:

- `custom_tokenizer(text, spec_tokens, pattern)`  
  Aplica una expresión regular que separa el texto en tokens, pero respeta ciertos tokens
  especiales (`spec_tokens`) que no deben partirse. Esto es útil para manejar etiquetas
  como contactos o marcadores de fin de mensaje.

- `get_vocab(text)`  
  Recorre todo el texto tokenizado y construye el **conjunto de tokens únicos**. Este
  conjunto se ordena y se usa como vocabulario.

- `encode(tokens, vocab)`  
  Toma una lista de tokens y un vocabulario, y devuelve una lista de índices enteros
  (por ejemplo `[15, 8, 203, 5, ...]`). Si un token no está en el vocabulario se reemplaza
  por `<UNK>`.

- `decode(indices, vocab)`  
  Realiza la operación inversa: dada una secuencia de índices, reconstruye la secuencia
  de tokens. Esto se usa tanto para depuración como para mostrar el texto generado por el modelo.




## 3. Construcción de batches autoregresivos

La construcción de los batches autoregresivos es fundamental para entrenar un modelo que aprenda a predecir el siguiente token en una secuencia. Primero, se toma el texto ya codificado como números y se divide en ventanas de tamaño fijo llamadas block_size. Cada ventana se utiliza de dos formas: como entrada (x), y como salida desplazada un paso hacia adelante (y). Por ejemplo, si el texto es [10, 20, 30, 40, 50] y el tamaño de ventana es 4, entonces x será [10, 20, 30, 40] y y será [20, 30, 40, 50].

Luego, la función que arma los batches elige varias posiciones dentro del texto y extrae desde allí estas ventanas de manera simultánea. Para cada posición seleccionada, se guarda una ventana como x y la misma ventana desplazada un token como y, formando así los pares con los que el modelo aprenderá. Esta extracción se hace cada vez que se necesita un batch, de modo que el modelo ve múltiples fragmentos distintos del corpus.

Finalmente, estos pares se envían al modelo durante el entrenamiento, permitiéndole aprender a predecir el siguiente token en cada posición. Este proceso repetido muchas veces garantiza que el modelo recorra completo en distintos contextos y aprenda la estructura del lenguaje de forma progresiva.


## 4. Arquitectura del modelo GPT (explicada con el código)

El modelo está basado en un Transformer tipo decodificador. En términos simples, su función es tomar una secuencia de tokens y aprender a predecir el siguiente. Todo esto se implementa en `model.py`, donde cada clase corresponde a una parte fundamental del modelo. A continuación explico las piezas principales con un lenguaje cercano y citando partes del código donde es necesario.



### 4.1 Atención de una sola cabeza

La clase `Head` implementa la atención. El objetivo es que cada token decida cuánto debe “mirar” a los tokens anteriores. En el código se crean las transformaciones de query, key y value:

```python
self.key = nn.Linear(embed_size, head_size, bias=False)
self.query = nn.Linear(embed_size, head_size, bias=False)
self.value = nn.Linear(embed_size, head_size, bias=False)
```

Luego, en el `forward`, se calcula la atención con:

```python
wei = q @ k.transpose(-2,-1)
wei = wei.masked_fill(tril == 0, float("-inf"))
```

Con esto, el modelo mide similitudes entre tokens y usa la máscara para no acceder al futuro. Finalmente combina los valores `v` según estos pesos. Esta parte permite que el modelo elija qué información previa es más útil.



### 4.2 Atención multi-cabeza

La clase `MultiHeadAttention` simplemente ejecuta varias cabezas como la anterior en paralelo:

```python
heads_list = [Head(head_size) for _ in range(n_heads)]
self.heads = nn.ModuleList(heads_list)
```

Cada cabeza aprende un tipo distinto de relación dentro del texto. Luego, las salidas se concatenan y se mezclan:

```python
out = torch.cat(heads_list, dim=-1)
out = self.linear(out)
```

Esto ayuda al modelo a entender patrones más variados y complejos.



### 4.3 Feed-forward por posición

Después de la atención, cada token pasa por una red totalmente conectada definida en `FeedFoward`:

```python
self.net = nn.Sequential(
    nn.Linear(embed_size, 4 * embed_size),
    nn.ReLU(),
    nn.Linear(4 * embed_size, embed_size),
    nn.Dropout(dropout),
)
```

Esta red no mezcla información entre posiciones; transforma cada vector por separado. Sirve para refinar la información que salió de las cabezas de atención.



### 4.4 Bloque Transformer completo

El bloque completo combina atención, MLP, normalización y conexiones residuales. En el `forward` vemos:

```python
x = x + self.sa(self.ln1(x))
x = x + self.ffwd(self.ln2(x))
```

Este patrón permite que el modelo agregue nueva información sin perder la representación original, haciendo que el entrenamiento sea más estable y permitiendo apilar varios bloques.



### 4.5 Modelo completo GPTLanguageModel

La clase `GPTLanguageModel` integra todo. Primero crea embeddings:

```python
self.token_embedding = nn.Embedding(vocab_size, embed_size)
self.pos_embedding = nn.Embedding(block_size, embed_size)
```

Cada token y cada posición tienen su propio vector. Luego se combinan:

```python
x = tok_emb + pos_emb
x = self.blocks(x)
logits = self.linear_output(x)
```

Los logits finales indican qué token es más probable como siguiente. Si se entregan `targets`, se calcula la pérdida usando entropía cruzada para guiar el aprendizaje.

El método `generate` muestra cómo el modelo escribe texto: predice probabilidades con softmax, elige un token mediante sampling con:

```python
idx_next = torch.multinomial(probs, num_samples=1)
```

y repite el proceso hasta encontrar el token de fin. De esta forma, el modelo va generando texto paso a paso.




## 5. Función de pérdida y entrenamiento (`train.py`)

### 5.1 Pérdida: entropía cruzada

En el método `forward` del modelo, cuando se entregan `targets`, se realiza:

1. Se reacomodan los logits de forma `(batch_size * time_steps, vocab_size)`.
2. Se reacomodan los targets de forma `(batch_size * time_steps,)`.
3. Se aplica `F.cross_entropy(logits, targets)`.

Esta función compara la distribución de probabilidades que el modelo asigna al siguiente token
(con una softmax implícita) con el token real observado en el corpus. Mientras más lejos estén,
mayor será la pérdida; el entrenamiento busca minimizar esta cantidad.

### 5.2 Bucle de entrenamiento

En `train.py` se hace lo siguiente (en pseudocódigo):

1. Cargar los tensores `train.pt` y `valid.pt` y el vocabulario.
2. Crear el modelo `GPTLanguageModel` con los hiperparámetros de `config.py`.
3. Definir el optimizador `AdamW` con una tasa de aprendizaje y un peso de decaimiento.
4. Para un número fijo de iteraciones (`max_iters`):
   - Obtener un batch de `x, y` desde `train` mediante `get_batch`.
   - Poner el modelo en modo entrenamiento (`model.train()`).
   - Calcular `logits` y `loss = model(x, y)`.
   - Hacer `optimizer.zero_grad()`, `loss.backward()` y `optimizer.step()`.
   - Periódicamente, evaluar el modelo en `train` y `valid` usando `estimate_loss`,
     que pone el modelo en modo evaluación (`model.eval()`) y promedia la pérdida
     sobre varios batches.

5. Guardar el modelo entrenado en `assets/models/model.pt` para usarlo después.




## 6. Generación de texto (modo chat, `model.generate` y `chat.py`)

El archivo model.py implementa un Transformer de tipo decodificador, que es el núcleo del modelo GPT. La clase Head define la atención de una sola cabeza, donde cada token del contexto decide qué tanto debe mirar a los tokens anteriores. Esto se realiza generando matrices query, key y value a partir de la entrada, calculando similitudes con q @ k.transpose(-2,-1) y aplicando una máscara para impedir acceder al futuro. Esta operación permite que el modelo identifique qué partes de la secuencia previa son relevantes para entender la posición actual.

Luego se define la clase MultiHeadAttention, que ejecuta varias cabezas de atención en paralelo para capturar distintos patrones del texto al mismo tiempo. Cada cabeza aprende relaciones diferentes y sus salidas se concatenan y mezclan mediante una capa lineal. Después aparece la clase FeedFoward, que aplica una pequeña red neuronal posición por posición para transformar la información obtenida por la atención. Estas dos partes, junto con normalización y conexiones residuales, se combinan dentro de la clase Block, donde se observa el patrón x = x + self.sa(...) y x = x + self.ffwd(...), lo que ayuda al modelo a estabilizar el entrenamiento y mantener información útil a lo largo de la red.

Finalmente, la clase GPTLanguageModel une todos estos componentes. Primero crea embeddings para representar tokens y posiciones, los suma para formar la entrada inicial y los pasa por varios bloques Transformer definidos en self.blocks. Luego una capa lineal final produce los logits que permiten predecir el siguiente token. Si se entregan objetivos, se calcula la pérdida con entropía cruzada; y en el modo de generación, el modelo toma el último logit, aplica softmax y selecciona un token con torch.multinomial, repitiendo este proceso hasta encontrar el token de fin. Esto permite que el modelo genere texto autoregresivamente, construyendo una secuencia palabra por palabra.
